In [ ]:
!pip install datasets
!pip install google-generativeai
!pip install --upgrade together


In [ ]:
!pip install wikipedia
!pip install sentence_transformers

In [14]:
!pip3 install text2num
!pip install wikipedia
!pip install word2number

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5567 sha256=c1e02193e5afa9ef6d2dc5c4ad03dbafdd582292091819cfe2d934fbaa8fc949
  Stored in directory: /home/anushkayadav_umass_edu/.cache/pip/wheels/84/ff/26/d3cfbd971e96c5aa3737ecfced81628830d7359b55fbb8ca3b
Successfully built word2number


In [1]:
import re
import pandas as pd
#from datasets import load_dataset
import time
import pickle
from tqdm import tqdm

### CHECK GENERATED DATA

In [7]:
df1=pd.read_pickle("anushka_generated_dataset.pkl")
df2=pd.read_pickle("arushi_generated_dataset_new_130.pkl")
df3=pd.read_pickle("arushi_generated_dataset_new_375.pkl")
df = pd.concat([df1, df2, df3], axis=0)
df.reset_index(drop=True, inplace=True)

In [9]:
df.tail()

,Q,A,W,C
996,What film title does J. F. Lawton and Shawn Le...,producer,"J. F. Lawton > Jonathan Frederick ""J. F."" Lawt...","First, identify the [film titles of J. F. Lawt..."
997,Jolen is a member of the race of what type of ...,superhumans,Jolen (comics) > He is a member of the Inhuma...,"First, find out [Jolen affiliation in comics -..."
998,What is the name of the album that was release...,Blackstar,"Death of David Bowie > On 10 January 2016, Eng...","First, find out [album released by David Bowie..."
999,What year was the Montgomery based school that...,1867,Mabel Murphy Smythe-Haith > He later moved to...,"First, determine the [school founded in Montgo..."
1000,"As of 2016, which band released more albums, T...",The Classic Crime,The Last Shadow Puppets > Following a lengthy...,"First, find out [albums released by The Last S..."


In [10]:
df.shape

(1001, 4)

In [15]:
import requests
import json
from bs4 import BeautifulSoup
from pprint import pprint
import wikipedia

from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

from transformers import pipeline
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')


def wikipedia_seach(query):
  language_code = 'en'

  number_of_results = 3
  headers = {
    # 'Authorization': 'Bearer YOUR_ACCESS_TOKEN',
    'User-Agent': 'YOUR_APP_NAME (YOUR_EMAIL_OR_CONTACT_PAGE)'
  }

  base_url = 'https://api.wikimedia.org/core/v1/wikipedia/'
  endpoint = '/search/page'
  url = base_url + language_code + endpoint

  search_query = query
  parameters = {'q': search_query, 'limit': number_of_results}
  response = requests.get(url, headers=headers, params=parameters)
  response = json.loads(response.text)

  searched_titles=[]
  for page in range(len(response['pages'])):
    display_title = response['pages'][page]['title']
    searched_titles.append(display_title)
    response['pages'][page]["article_url"] = 'http://en.wikipedia.org/?curid=' + str(response['pages'][page]['id'])
    response['pages'][page]['excerpt'] = BeautifulSoup(response['pages'][page]['excerpt']).get_text()

  excerpt_text=search_query+": "+"\n".join([page['excerpt'] for page in response['pages']])
  if(len(searched_titles)!=0):
    #go to page whose title+ excerpt is most similar
    # Encode search query
    query_embedding = sentence_model.encode(search_query)
    page_texts = [page['title'] + ": " + page['excerpt'] for page in response['pages']]
    page_embeddings = sentence_model.encode(page_texts)
    # Calculate similarity scores for each page
    similarities = cosine_similarity([query_embedding], page_embeddings)[0]

    title_score_dict = dict(zip(searched_titles, similarities))

    # Get the index of the page with the highest similarity
    max_similarity_index = similarities.argmax()

    # Output the page id with the highest similarity
    max_similarity_page_id = response['pages'][max_similarity_index]['id']

    try:
      a=wikipedia.page(pageid=max_similarity_page_id)
      full_content=a.content
      return title_score_dict,excerpt_text,full_content
    except:
      return {},"",""
  else:
    return {},"",""

def QA(ques, context):
  result = question_answerer(question=ques,context=context)
  return result['answer'], result['score']

In [16]:
import requests
import json
import numpy as np
from text_to_num import text2num

url = "https://google.serper.dev/search"

def google_search(query):
  payload = json.dumps({
  "q": query
})
  headers = {
  'X-API-KEY': '44218db214e14ddca03d1a19c15af17344e35dcf',
  'Content-Type': 'application/json'
  }
  excerpt_text = ""
  full_content = ""
  most_similar_snippets = ""
  title_score_dict = {}


  try:
    response = requests.request("POST", url, headers=headers, data=payload)
    response = json.loads(response.text)
    #print(response)
    if 'answerBox' in response:
      if 'snippetHighlighted' in response['answerBox']:
        excerpt_text = response['answerBox']['snippetHighlighted'][0]
      elif 'answer' in response['answerBox']:
        excerpt_text = response['answerBox']['answer']
      else:
        excerpt_text = response['answerBox']['snippet']
      
    else:
      #print("HERE")
      full_content = [results['title'] + ": " + results['snippet'] for results in response['organic']]

    if full_content:
      query_embedding = sentence_model.encode(query)
      snippet_embeddings = sentence_model.encode(full_content)
      similarities = cosine_similarity([query_embedding], snippet_embeddings)[0]
      snippets_greater_than_70 = np.where(similarities > 0.50)[0]
      most_similar_snippets = " ".join([full_content[index] for index in snippets_greater_than_70])

    return title_score_dict,excerpt_text,most_similar_snippets
  except:
    return {},"",""

In [ ]:
## CHECKING STEP

In [ ]:
# first level scoring on wikipedia search
#is wikipedia top3 matching the titles in W
#if answer not found in excerpts , go to main wikipedia page contet whose title/excerpt matches highest

In [21]:
row=2
print(df.iloc[row]['Q'])
print(df.iloc[row]['W'])
print(df.iloc[row]['C'])
print(df.iloc[row]['A'])

Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
Allie Goertz > Allison Beth "Allie" Goertz (born March 2, 1991) is an American musician. | Allie Goertz >  Goertz is known for her satirical songs based on various pop culture topics. | Allie Goertz >  Her videos are posted on YouTube under the name of Cossbysweater. | Milhouse Van Houten > Milhouse Mussolini van Houten is a fictional character featured in the animated television series "The Simpsons", voiced by Pamela Hayden, and created by Matt Groening who named the character after President Richard Nixon's middle name.
First, identify [Matt Groening named Milhouse after -Wiki-> y1]. The name is [y1 -QA(Who did Matt Groening name Milhouse after?)-> y2]. The answer is y2.
President Richard Nixon


In [22]:
def process_list_tuple(rep): ## DECISION HERE
  if(str(type(rep[0]))!="<class 'tuple'>"): #if wikipedia article comes as in query, return empty string as it is anyways invalid
    return ""
  #for QA result, best score string returned
  non_empty_rep = [item for item in rep if item]  # Filter out empty tuples
  if non_empty_rep:
    rep = max(non_empty_rep, key=lambda x: x[1])[0]  # Get the string with the highest score
  else:
    rep = ""  # If all tuples are empty, return an empty string
  return str(rep)

def get_variables(text):
  matches = re.findall(r'\[(.*?)\]', text)
  variables={}
  wiki_searches=[]
  errors=[]
  variables['all_eqs']=[]

  def replace_variable(match):
    #print(variables)
    rep = variables.get(match.group(), match.group())
    if isinstance(rep, list):
        rep=process_list_tuple(rep)
    return rep

  for instruction in matches:
    #print('at ', instruction)
    #print(variables)
    #---WIKI TOOL---
    if(instruction.find("-Wiki")!=-1):
      query,ans=instruction.split(" -Wiki-> ") #for in query variables
      in_query_var=re.search(r'y\d+', query)
      if(in_query_var):
        query=re.sub(r'y\d+', replace_variable, query)

      title_score_dict,excerpt_text,full_content=google_search(query)
      #forcing wiki here
      #excerpt_text=''
      #full_content=''
      if not excerpt_text and not full_content:
        print('WIKI SEARCH')
        title_score_dict,excerpt_text,full_content=wikipedia_seach(query)#wikipedia search
        if title_score_dict: #SAVING IN VARIABLES HERE
          #print('HERE updating varibale of wiki answer',ans.strip())
          variables[ans.strip()] = [excerpt_text, full_content]#replaces the answer variable with extracted content of wiki
          wiki_searches.append(title_score_dict)
        else:
          errors.append(f"Error: Wikipedia search unsuccessful for query: '{query}'")
          break
      else:
        variables[ans.strip()] = [excerpt_text, full_content]#replaces the answer variable with extracted content of google 

    #---QA TOOL---
    elif(instruction.find("-QA")!=-1):
      q,a=instruction.split("->")
      result = re.search(r'\((.*?)\)', q)
      if result:
          question = result.group(1)
      else:
          errors.append("Error in finding question")
          continue

      pre=q.split(" -QA")[0]
      context_exc=""
      full_context=""
      if (pre.find("+")!=-1): #if multiple contexts need to be appended

        vars=pre.split("+")
        for i in vars:
          if(i.strip() in variables.keys()):
            if((str(type(variables[i.strip()][0]))!="<class 'tuple'>")):
              context_exc=variables[i.strip()][0] #if variable came from wiki search, it would be a string else it would be a tuple
              full_context=variables[i.strip()][1]
            else:
              t=process_list_tuple(variables[i.strip()])
              #print('-----t',t)
              context_exc=t #if variable came from wiki search, it would be a string else it would be a tuple
              full_context=t
          else:
            errors.append(f"Error: Context variable '{i.strip()}' not found")
            break

      else: #single context QA
        context_var=pre.strip()
        if context_var in variables.keys():
          if((str(type(variables[context_var][0]))!="<class 'tuple'>")):
            context_exc=variables[context_var][0] #if variable came from wiki search, it would be a string else it would be a tuple
            full_context=variables[context_var][1]
          else:
            t=process_list_tuple(variables[context_var])
            context_exc=t #if variable came from wiki search, it would be a string else it would be a tuple
            full_context=t
        else:
          errors.append(f"Error: Context variable '{context_var}' not found")
          break

      #cant send empty context to QA model
      if(context_exc!=""):
        #print(question,context_exc)
        execerpt_run=QA(question, context_exc)
        if(execerpt_run[-1]<0.75 and len(full_context)>0):
          full_run=QA(question, full_context)
        else:
          full_run=()
      else:
        execerpt_run=()
        if(len(full_context)>0):
          full_run=QA(question, full_context)
        else:
          full_run=()


      #print('Updating variable',a.strip())
      variables[a.strip()]=[execerpt_run,full_run]

    #----MATH TOOL----
    else:
      #print('going to math tool')
      #print('at math',instruction)
      variables['all_eqs'].append(instruction)
      equation_variables = re.findall(r'y\d+', instruction)
      #print('found varibales',equation_variables)
      for var in equation_variables:
        if(var in variables.keys()):
          #print('on', var)
          var_ans=process_list_tuple(variables[var])
          #print('before process',variables[var])
          #print('on process',var_ans)
          numbers = re.findall(r'\d+(?:\.\d+)?', var_ans)#we try searching for numbers
          if(len(numbers)>0):
            #print('adding equation',var, numbers)
            variables['all_eqs'].append(var+" = "+str(numbers[0])) #making equation for previously retrieved numerical values ### DECISON HERE
            errors.append(f"WARNING: '{str(numbers)}'numbers in var ans")
          else:
            try:
              #var_ans=str(text2num(var_ans, "en"))
              var_ans=str(w2n.word_to_num(var_ans))
              numbers = re.findall(r'\d+(?:\.\d+)?', var_ans)#we try searching for numbers
              if(len(numbers)>0):
                #print('adding equation',var, numbers)
                variables['all_eqs'].append(var+" = "+str(numbers[0])) #making equation for previously retrieved numerical values ### DECISON HERE
                errors.append(f"WARNING: '{str(numbers)}'numbers in var ans")
            except:
              print('var ans is not a numeric',var_ans)
              errors.append(f"Error: '{var_ans}' not numeric")


  return variables,wiki_searches,errors


In [25]:
def solve_all(equations):
  ans_dict={}
  list_eq=[]
  for i in equations:
    eq=sympify("Eq(" + i.replace("=", ",") + ")")
    list_eq.append(eq)
  #print(list_eq)
  result =solve(list_eq)
  #print(result)
  if(str(type(result))=="<class 'list'>"):
    result=result[0]
  for i in result.keys():
      ans_dict[str(i)]=result[i]
  return ans_dict

In [26]:
row=1
print(df.iloc[row]['Q'])
print(df.iloc[row]['C'])
print(df.iloc[row]['A'])
text=df.iloc[row]['C']
matches = re.findall(r'\[(.*?)\]', text)
matches

The Oberoi family is part of a hotel company that has a head office in what city?
Search [Oberoi Group head office city -Wiki-> y1]. The city is [y1 -QA(The Oberoi Group head office is in which city?)-> y2]. The answer is y2.
Delhi


['Oberoi Group head office city -Wiki-> y1',
 'y1 -QA(The Oberoi Group head office is in which city?)-> y2']

In [27]:
v,w,e=get_variables(text)

In [28]:
v

{'all_eqs': [],
 'y1': ['New Delhi, India', ''],
 'y2': [('New Delhi', 0.984399676322937), ()]}

In [29]:
processed=[]

In [30]:
len(processed)

0

In [31]:

for row in tqdm(range(df.shape[0])):
  row_d=df.iloc[row].to_dict()
  try:
    answer_var=row_d['C'].lower().split("answer is")[-1].replace(".","").strip()
    variables, wiki_searches, errors = get_variables(row_d['C'])
    if(len(variables['all_eqs'])>0):
      #solve eqs
      try:
        variables.update(solve_all(variables['all_eqs']))
      except:
        errors.append("Equaltions not solvable")
    if(answer_var in variables.keys()):
      answer=variables[answer_var]
    else:
      answer="not found"
  except:
    variables=[]
    wiki_searches=[]
    errors=['Outer Error Handling']
    answer="not found"

  row_d['Variables'] = variables
  row_d['Searches'] = wiki_searches
  row_d['Errors'] = errors
  row_d['Generated_answer'] = answer
  processed.append(row_d)

  # Save to pickle file every 50 rows
  if row % 50 == 0 and row != 0:
      processed_df = pd.DataFrame(processed)
      #processed_df.to_pickle(f'processed_rows_wiki2.pkl')


# Save any remaining processed rows to a pickle file
if(processed):
    processed_df = pd.DataFrame(processed)
    #processed_df.to_pickle(f'processed_rows_final_wiki2.pkl')

  6%|▋         | 64/1001 [01:26<24:40,  1.58s/it]

WIKI SEARCH


  9%|▊         | 87/1001 [02:02<19:02,  1.25s/it]

WIKI SEARCH


  9%|▉         | 94/1001 [02:12<20:00,  1.32s/it]

...

 76%|███████▋  | 765/1001 [18:04<06:58,  1.77s/it]

WIKI SEARCH


 81%|████████  | 807/1001 [19:04<05:38,  1.74s/it]

var ans is not a numeric Eugene Lee
Ragtime
var ans is not a numeric Alexandra Shiva
var ans is not a numeric Alexandra Shiva


100%|██████████| 1001/1001 [23:30<00:00,  1.41s/it]


In [42]:
row=3
print(df.iloc[row]['Q'])
print(df.iloc[row]['C'])
print(df.iloc[row]['A'])
text=df.iloc[row]['C']
matches = re.findall(r'\[(.*?)\]', text)
matches

 What nationality was James Henry Miller's wife?
Find out [James Henry Miller wife nationality -Wiki-> y1]. Her nationality is [y1 -QA(What nationality was James Henry Miller's wife?)-> y2]. The answer is y2.
American


['James Henry Miller wife nationality -Wiki-> y1',
 "y1 -QA(What nationality was James Henry Miller's wife?)-> y2"]

In [41]:
processed_df.iloc[3]['Variables']

{'all_eqs': [],
 'y1': ['',
  'Henry Miller - Wikipedia: In 1967, Miller married his fifth wife, Japanese born singer Hoki Tokuda (ja:ホキ徳田). James Henry Miller (1860–1935) • FamilySearch: James Henry Miller was born on 20 June 1860, in Alabama, United States as the son of Mr Miller and Mrs Miller. He married Nancy Ophelia Moore on 10 April ... James Henry Miller Obituary - Ventura County Star: James (Jim) H. Miller, loving husband to Carol Miller, passed away on February 15th. He was born in 1932 to James and Esther Miller. Cpl. James Henry Miller (USA) - Geni.com: James was the son of Joseph Miller and Nancy Jane Miller. He married Martha Simon on Mar 6 862 in Buckhannon, West Virginia. This marriage ... Obituary for James Henry Miller | Henline - Hughes Funeral Home: James Henry Miller, age 71 of Hwy 226 North, Bakersville, NC passed away August 25, 2014 at his home. He was a native of Mitchell County, NC and ... James Henry Miller living in Southwark, London,Surrey in 1911 ...: Mart

In [39]:
processed_df.head()

,Q,A,W,C,Variables,Searches,Errors,Generated_answer
0,Which magazine was started first Arthur's Maga...,Arthur's Magazine,Arthur's Magazine > Arthur's Magazine (1844–18...,"First, search [Arthur's Magazine -Wiki-> y1]. ...","{'all_eqs': [], 'y1': ['', 'Arthur's Magazine ...",[],[],"[(), (First For Women - Health, Diet, Beauty T..."
1,The Oberoi family is part of a hotel company t...,Delhi,Oberoi family > The Oberoi family is an Indian...,Search [Oberoi Group head office city -Wiki-> ...,"{'all_eqs': [], 'y1': ['New Delhi, India', '']...",[],[],"[(New Delhi, 0.984399676322937), ()]"
2,Musician and satirist Allie Goertz wrote a son...,President Richard Nixon,"Allie Goertz > Allison Beth ""Allie"" Goertz (bo...","First, identify [Matt Groening named Milhouse ...","{'all_eqs': [], 'y1': ['U.S. president Richard...",[],[],"[(Richard Nixon, 0.982280969619751), ()]"
3,What nationality was James Henry Miller's wife?,American,"Peggy Seeger > Margaret ""Peggy"" Seeger (born J...",Find out [James Henry Miller wife nationality ...,"{'all_eqs': [], 'y1': ['', 'Henry Miller - Wik...",[],[],"[(), (Japanese, 0.6979333162307739)]"
4,Cadmium Chloride is slightly soluble in this c...,alcohol,Cadmium chloride > It is a hygroscopic solid ...,"First, search [Cadmium Chloride chemical it is...","{'all_eqs': [], 'y1': ['slightly soluble in al...",[],[],"[(alcohol, 0.9992592334747314), ()]"


In [43]:
processed_df.to_pickle("wikipedia_processed1000.pkl")

In [36]:
import pandas as pd
from fuzzywuzzy import fuzz
from sentence_transformers import SentenceTransformer, util
import torch
from torch import nn

# Define the additional functions for similarity checks
def exact_match(text1, text2):
    return text1 == text2

def fuzzy_match(text1, text2):
    ratio = fuzz.partial_ratio(text1.lower(), text2.lower())
    return ratio

def semantic_similarity(text1, text2):
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings1 = model.encode(text1, convert_to_tensor=True)
    embeddings2 = model.encode(text2, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
    return cosine_scores.item()

def semantic_similarity_phrase_bert(text1, text2):
    phrase_list = [text1, text2]
    model = SentenceTransformer('whaleloops/phrase-bert')
    phrase_embs = model.encode(phrase_list)
    [p1, p2] = phrase_embs
    cos_sim = nn.CosineSimilarity(dim=0)
    return cos_sim(torch.tensor(p1), torch.tensor(p2))


/home/anushkayadav_umass_edu/.conda/envs/llmal/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [44]:
# Update the check_answer function to include new similarity checks
def check_answer(row):
    gen = row['Generated_answer']
    act = row['A']
    all_vars = row['Variables']

    if exact_match(gen, act):
        return True
    if str(gen).find(str(act)) != -1:
        return True
    if str(gen) == str(act):
        return True
    if fuzzy_match(str(gen), act) > 80:  # You can adjust the threshold for fuzzy matching
        return True
    if semantic_similarity(str(gen), act) > 0.6:  # Adjust the threshold for semantic similarity
        return True
    if semantic_similarity_phrase_bert(str(gen), act) > 0.6:  # Adjust the threshold for phrase BERT similarity
        return True
    return False


In [46]:
from tqdm import tqdm

# Assuming 'processed_df' is your DataFrame
tqdm.pandas()  # Initialize tqdm with pandas

# Apply the check_answer function with a progress bar
processed_df['is_final_in_solved'] = processed_df.progress_apply(check_answer, axis=1)

100%|██████████| 1001/1001 [18:28<00:00,  1.11s/it]


In [47]:
processed_df['is_final_in_solved'].unique()

array([False,  True])

In [48]:
# Count the rows where final_answer is present in solved_answer list
count_valid = processed_df[processed_df['is_final_in_solved']].shape[0]

In [49]:
count_valid 

332

In [50]:
processed_df.to_pickle("wikipdia1000_validated.pkl")

In [ ]:
processed_df.tail(20)

In [59]:
# Define the words to filter for
error_words = ['Error']

# Function to check if any word in a list is in the specified error_words
def has_error_words(lst):
    return not any(word in str(lst) for word in error_words)

# Apply the function to filter rows
non_error_df = processed_df[processed_df['Errors'].apply(has_error_words)]
non_error_df.shape

(921, 9)

In [60]:
solved_rows_df = processed_df[processed_df['is_final_in_solved'] == True]
solved_rows_df = solved_rows_df[solved_rows_df['Errors'].apply(has_error_words)]
solved_rows_df.shape

(321, 9)

In [61]:
# Identify rows in non_error_df that are not in solved_rows_df
unique_non_error_df = non_error_df[~non_error_df.index.isin(solved_rows_df.index)]

# Select 40 rows from the identified rows
selected_rows = unique_non_error_df.sample(n=200, random_state=42)  # Adjust the sampling criteria as needed

# Concatenate selected rows with solved_rows_df
concatenated_df = pd.concat([solved_rows_df, selected_rows])

In [64]:
concatenated_df.shape

(521, 9)

In [65]:
concatenated_df.to_pickle("../all_data_finals/atFinal-wikitool.pkl")

# EXPERIMENT

In [11]:
row=95
text=df.iloc[row]['C']
matches = re.findall(r'\[(.*?)\]', text)
matches

['type of script used in autographs -Wiki-> y1',
 'y1 -QA(What script was used in autographs?)-> y2',
 'Cuneiform script invented by-Wiki-> y3',
 'y3 -QA(Who invented cuneiform script?)-> y4']

In [ ]:
def replace_variable(match):
    return variables.get(match.group(), match.group())

variables={}
wiki_searches=[]
for instruction in matches:
  #print("at isnt",instruction,)
  if(instruction.find("-Wiki")!=-1):
    query,ans=instruction.split(" -Wiki-> ")
    #for in query variables
    in_query_var=re.search(r'y\d+', query)
    if(in_query_var):
      print("FOUND INQUERY VAR")
      query=re.sub(r'y\d+', replace_variable, query)
    #wikipedia search
    title_score_dict,excerpt_text,full_content=wikipedia_seach(query)
    #replaces the answer variable with extracted content
    variables[ans.strip()]= [excerpt_text,full_content]
    wiki_searches.append(title_score_dict)

  if(instruction.find("-QA")!=-1):
    q,a=instruction.split("->")
    result = re.search(r'\((.*?)\)', text)
    if result:
        question = result.group(1)
    else:
        print("Error in finding question")
        continue

    pre=q.split(" -QA")[0]
    if (pre.find("+")!=-1):
      vars=pre.split("+")
      context_exc=""
      full_context=""
      for i in vars:
        if(i.strip() in variables.keys()):
          context_exc=context_exc+variables[i.strip()][0]
          full_context=full_context+variables[i.strip()][1]
        else:
          print('error in finding context variable')
          continue

    else:
      context_var=pre.strip()
      if context_var in variables.keys():
        context_exc=variables[context_var][0]
        full_context=variables[context_var][1]
      else:
        print('error in finding context variable')
        continue

    execerpt_run=QA(question, context_exc)
    if(execerpt_run[-1]<0.75):
      full_run=QA(question, full_context)
    else:
      full_run=()
    variables[a.strip()]=[execerpt_run,full_run]

at isnt Henri Leconte Grand Slam titles  -Wiki-> y1
at isnt Jonathan Stark Grand Slam titles  -Wiki-> y2
at isnt y1+y2 -QA(Which tennis player won more Grand Slam titles, Henri Leconte or Jonathan Stark?)-> y3


In [ ]:
variables

In [ ]:
wiki_searches

[{'Henri Leconte': 0.6307155,
  'Grand Slam Cup': 0.3890447,
  "Open Era tennis records – Men's singles": 0.378835},
 {'Jonathan Stark (tennis)': 0.7212882,
  'Serena Williams': 0.47569674,
  "List of Grand Slam men's doubles champions": 0.4702412}]

In [ ]:
true_wiki_pages=[i.split(">")[0].strip() for i in df.iloc[5]['W'].split("|")]
true_wiki_pages

['Jonathan Stark (tennis)', 'Jonathan Stark (tennis)', 'Henri Leconte']

In [ ]:
query="Number of legs in chicken"
t,e,f=wikipedia_seach(query)

37402
